In [7]:
import numpy as np
from scipy.stats import norm


class BlackModel:
    def d_1(self, t, F, K, sigma):
        return (1 / (sigma * np.sqrt(t))) * (np.log(F / K) + 0.5 * (sigma ** 2) * t)

    def d_2(self, t, F, K, sigma):
        return self.d_1(t, F, K, sigma) - sigma * np.sqrt(t)

    def put(self, r, t, F, K, sigma):
        return np.exp(-1 * r * t) * (K * norm.cdf(-1 * self.d_2(t, F, K, sigma)) -
                                     F * norm.cdf(-1 * self.d_1(t, F, K, sigma)))

    def call(self, r, t, F, K, sigma):
        return np.exp(-1 * r * t) * (F * norm.cdf(self.d_1(t, F, K, sigma)) -
                                     K * norm.cdf(self.d_2(t, F, K, sigma)))


def TrinomialPut(S0, K, r, sigma, T):
    
    N=360
    dt = T / N  # time steps
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = np.exp(-1 * sigma * np.sqrt(2 * dt))
    m = 1

    pu = ((np.exp(r * dt / 2) - np.exp(-1 * sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-1 * sigma * np.sqrt(dt / 2)))) ** 2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-1 * sigma * np.sqrt(dt / 2)))) ** 2
    pm = 1 - pu - pd

    STs = [np.array([S0])]

    for i in range(N):
        prev_nodes = STs[-1]
        ST = np.concatenate(
            (prev_nodes * u * np.exp(-1 * r * dt), [prev_nodes[-1] * m * np.exp(-1 * r * dt),
                                                    prev_nodes[-1] * d * np.exp(-1 * r * dt)]))
        STs.append(ST)

    payoffs = np.maximum(0, (K - STs[N]))

    for i in reversed(range(N)):
        payoffs = (payoffs[:-2] * pu + payoffs[1:-1] * pm + payoffs[2:] * pd) * np.exp(-1 * r * dt)

    return payoffs



In [8]:
blackModel = BlackModel()
print(blackModel.put(r=0.015, t=0.2639, K=2900, F=3120, sigma=0.175))
   

31.602344585498123


In [9]:
print(TrinomialPut(S0=3120, K=2900, r=0.015, sigma=0.175, T=0.2639))

[31.61724102]
